In [ ]:
!pip install Orange3

In [18]:
import Orange

# Rules - cn2

## Base risco de crédito

In [3]:
base_risco_credito = Orange.data.Table('/content/drive/MyDrive/datasets/risco_credito_regras.csv')

In [4]:
base_risco_credito

[[ruim, alta, nenhuma, 0_15 | alto],
 [desconhecida, alta, nenhuma, 15_35 | alto],
 [desconhecida, baixa, nenhuma, 15_35 | moderado],
 [desconhecida, baixa, nenhuma, acima_35 | alto],
 [desconhecida, baixa, nenhuma, acima_35 | baixo],
 ...
]

In [7]:
# Mostrando as colunas
base_risco_credito.domain

[historia, divida, garantias, renda | risco]

In [8]:
# Criando a variável que recebe o algoritmo cn2
cn2 = Orange.classification.rules.CN2Learner()

In [9]:
# Criando as regras
regras_risco_credito = cn2(base_risco_credito)

In [10]:
# Printando as regras criadas
for regras in regras_risco_credito.rule_list:
    print(regras)

IF renda==0_15 THEN risco=alto 
IF historia==boa AND divida!=alta THEN risco=baixo 
IF historia==boa AND garantias!=nenhuma THEN risco=baixo 
IF historia==boa AND renda!=15_35 THEN risco=baixo 
IF historia==boa THEN risco=moderado 
IF divida==alta THEN risco=alto 
IF historia!=desconhecida THEN risco=moderado 
IF garantias==adequada THEN risco=baixo 
IF renda==15_35 THEN risco=moderado 
IF historia==desconhecida THEN risco=alto 
IF TRUE THEN risco=alto 


In [11]:
# Fazendo testes
# história boa, dívida alta, garantias nenhuma, renda > 35
# história ruim, dívida alta, garantias adequada, renda < 15

previsoes = regras_risco_credito([['boa', 'alta', 'nenhuma', 'acima_35'], ['ruim', 'alta', 'adequada', '0_15']])
previsoes

array([1, 0])

In [14]:
base_risco_credito.domain.class_var

DiscreteVariable(name='risco', values=('alto', 'baixo', 'moderado'))

In [15]:
for i in previsoes:
    print(base_risco_credito.domain.class_var.values[i])

baixo
alto


## Base credit data - 97.40%

In [22]:
# Criando a variável que contém a base de dados já preparada
base_credit = Orange.data.Table('/content/drive/MyDrive/datasets/credit_data_regras.csv')

In [23]:
# Verificando as colunas
base_credit.domain

[income, age, loan | default]

In [24]:
# Dividindo a base de dados em duas
base_dividida = Orange.evaluation.testing.sample(base_credit, n=0.25) # Dividindo uma parte em 75% e outra em 25%

In [26]:
base_dividida

([[54738.7, 26.7719, 6210.73 | 0],
  [30218.2, 48.0215, 5914.52 | 0],
  [53298.5, 56.9097, 2106.71 | 0],
  [28982.1, 39.0484, 2898.76 | 0],
  [46766.6, 47.7752, 2383.41 | 0],
  ...
 ],
 [[39217.9, 30.1014, 864.624 | 0],
  [51903.5, 59.7627, 8807.87 | 0],
  [43372.4, 55.1946, 474.525 | 0],
  [24709.1, 54.7452, 1915.39 | 0],
  [39685, 33.6289, 2590.93 | 0],
  ...
 ])

In [28]:
# Pegando a parte de treino e teste
base_credit_treinamento = base_dividida[1]
base_credit_teste = base_dividida[0]
print(f'Tenho {len(base_credit_treinamento)} registros para treinar.')
print(f'Tenho {len(base_credit_teste)} registros para testar.')

Tenho 1500 registros para treinar.
Tenho 500 registros para testar.


In [29]:
# Criando o algoritmo
cn2 = Orange.classification.rules.CN2Learner()
regras_credit = cn2(base_credit_treinamento)

In [31]:
# Mostrando a tabela de regras
for regras in regras_credit.rule_list:
    print(regras)

IF age>=34.851817262359 THEN default=0 
IF age>=34.795262857340305 THEN default=1 
IF age>=34.5552007704973 THEN default=0 
IF age>=34.511520112399 THEN default=1 
IF loan<=2495.13299137587 AND income>=20145.9885970689 THEN default=0 
IF loan>=7718.479795185201 AND age>=20.8043047981337 THEN default=1 
IF income>=47683.715778403 AND loan>=8092.982779946221 THEN default=1 
IF age>=34.4421687847935 THEN default=1 
IF age>=34.1997820041142 THEN default=0 
IF income>=53493.4860118665 AND income>=58168.474065605595 THEN default=0 
IF loan>=6019.83442285853 AND age>=23.1489815721288 THEN default=1 
IF loan>=6415.0862444378 THEN default=1 
IF income>=48790.1324336417 THEN default=0 
IF income<=31702.3342987522 AND loan>=3665.88089899456 THEN default=1 
IF loan>=5416.3577980789005 AND loan>=5659.509277874139 THEN default=1 
IF loan>=5416.3577980789005 AND loan>=5650.88968761866 THEN default=0 
IF loan>=5416.3577980789005 THEN default=1 
IF income>=26617.0303151011 AND income>=40614.7220451385 

In [32]:
# Fazendo previsoes
previsoes_credit = Orange.evaluation.testing.TestOnTestData(base_credit_treinamento, base_credit_teste, [lambda testdata: regras_credit])

In [33]:
previsoes_credit

In [34]:
Orange.evaluation.CA(previsoes_credit)

array([0.976])

# Majority learner - classificador base

## Base credit data

In [35]:
base_risco_credito = Orange.data.Table('/content/drive/MyDrive/datasets/risco_credito_regras.csv')

In [36]:
base_credit.domain

[income, age, loan | default]

In [38]:
majority = Orange.classification.MajorityLearner()
# Fazendo previsões
previsoes = Orange.evaluation.testing.TestOnTestData(base_credit, base_credit, [majority]) # Treinando e testando na mesma base de dados

In [39]:
Orange.evaluation.CA(previsoes)

array([0.8585])

In [41]:
for registro in base_credit:
    print(registro.get_class())

0
0
0
0
1
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
1
0
0
0
1
1
0
0
0
0
0
0
0
0
0
0
0
1
1
1
0
0
0
0
0
0
0
0
0
0
0
1
1
0
0
0
1
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
1
0
1
1
0
1
0
0
1
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
1
0
1
0
0
0
0
0
0
0
0
0
0
1
0
1
1
0
0
0
1
1
0
0
0
0
1
0
0
0
0
1
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
1
1
0
0
0
1
0
1
0
0
0
0
0
0
0
0
1
0
0
0
1
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
1
0
1
0
1
0
1
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
1
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
1
0
0
0
1
0
0
0
1
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
1
0
0
1
0
0
0
0
0
0
1
0
0
1
0
0
0
0
0
1
0
0
0
0
0
1
0
1
0
0
0
1
0
1
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
1
0
1
0
0
1
1
0
0
0
0
0
1
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
1
0
1
0
0
0
0
0
1
0
0
0
1
0
0
1
0
0
0
0


In [42]:
from collections import Counter
Counter(str(registro.get_class()) for registro in base_credit)

Counter({'0': 1717, '1': 283})

## Base census